# K8s Provider and Universe

### Setup

First, import the required packages.

Next, specify some important variables:
* `platform_address`: The address of the Yugabyte Platform API
* `platform_api_key`: The API key used to authenticate with the Platform API

Finally, open a HTTP connection to the Yugabyte Platform API.

In [ ]:
import os
import http.client
import json
import yaml
from pprint import pprint

platform_address = os.getenv('API_BASE_URL', "localhost:9000")
#platform_api_key = os.getenv('YB_API_KEY')
platform_api_key = "3002b2d7-50a3-40fb-9eec-4b1360da73e7"

conn = http.client.HTTPConnection(f"{platform_address}")

headers = {
  'Content-Type': "application/json",
  'X-AUTH-YW-API-TOKEN': f"{platform_api_key}"
}


### Get Session Info

Make an API call to session endpoint to determine customer UUID.

In [ ]:
conn.request("GET", "/api/v1/session_info", headers=headers)

res = conn.getresponse()
data = res.read()
session_info = json.loads(data)
customer_uuid = session_info['customerUUID']

print('Customer UUID:\n%s' % customer_uuid)

In [ ]:
secret_filename = os.getenv('K8S_PULL_SECRET_FILE', "/home/sbapat/downloads/yugabyte-k8s-secret.yml")
with open(secret_filename) as f:
    secret = yaml.safe_load(f)

secret_name = secret['metadata']['name']
# print(secret_name)

kubeconfig_filename = os.getenv('K8S_KUBECONFIG', "/tmp/yugabyte-platform-universe-management.conf")
kubeconfig_file = open(kubeconfig_filename, 'r')
provider_overrides = \
'''
resource:
  master:
    requests:
      cpu: 2
      memory: 2Gi
    limits:
      cpu: 2
      memory: 2Gi
  tserver:
    requests:
      cpu: 2
      memory: 4Gi
    limits:
      cpu: 2
      memory: 4Gi
'''
new_provider = {
    "code": "kubernetes",
    "config": {
        'KUBECONFIG_IMAGE_PULL_SECRET_NAME': secret_name,
        'KUBECONFIG_PULL_SECRET_NAME': secret_name,
        'KUBECONFIG_PULL_SECRET_CONTENT': json.dumps(secret),
        "KUBECONFIG_IMAGE_REGISTRY":"quay.io/yugabyte/yugabyte",
    },
    "name": "my-gke-provider",
    "regions": [{'code': 'us-west1',
                 'name': 'us-west1',
                 'latitude': 0.0,
                 'longitude': 0.0,
                 'zones': [{'code': 'us-west1-b',
                               # 'config': {'STORAGE_CLASS': 'standard'},
                               'name': 'us-west1-b'},
                              {'code': 'us-west1-c',
                               'name': 'us-west1-c',
                               'name': 'my-zone',
                               'config': {
                                 'name': 'default',
                                 'KUBECONFIG_CONTENT': kubeconfig_file.read(),
                                 'KUBECONFIG_NAME': kubeconfig_filename,
                                 'OVERRIDES' : provider_overrides
                               },
                              },
                              {'code': 'us-west1-a',
                               # 'config': {'STORAGE_CLASS': 'standard'},
                               'name': 'us-west1-a'}]}],
}
#pprint(new_provider)

In [ ]:
url = f"/api/v1/customers/{customer_uuid}/providers"
conn.request("POST", url, json.dumps(new_provider), headers)
res = conn.getresponse()
data = res.read()
print(data)

### Get Provider UUID By Name

Make an API call to the provider endpoint to determine provider UUID and regions.

In [ ]:
provider_name = 'my-gke-provider' # my-gcp-rovider 
url = f"/api/v1/customers/{customer_uuid}/providers"
conn.request("GET", url, headers=headers)
res = conn.getresponse()

provider_list = json.load(res)
for provider in provider_list:
    if provider['name'] == provider_name and provider['active'] == True:
        provider_uuid = provider['uuid']
        region_list = [region['uuid'] for region in provider['regions']]
        break

print('Provider UUID:\n%s' % provider_uuid)
print('Regions:\n[%s]' % ', '.join(region_list))

### Get Instance Types Available

Make an API call to see the instance types available for the provider.

In [ ]:
url = f"/api/v1/customers/{customer_uuid}/providers/{provider_uuid}/instance_types"
conn.request("GET", url, headers=headers)
res = conn.getresponse()

instance_type = None
for i in json.load(res):
    print(i['instanceTypeCode'])
    if i['instanceTypeCode'] in ['large']: # desired aws or gcp type code
        print('^^^^^^^ Found !!!!!!!!!!!')
        instance_type = i
        break

pprint(instance_type['instanceTypeDetails'])

In [ ]:
new_universe = {
    'clusters': [
        {
            'clusterType': 'PRIMARY',
            'userIntent': {
                'universeName': 'my-k8s-universe', # or my-gcp-universe
                'providerType': 'kubernetes',
                'provider': provider_uuid,
                'regionList': region_list,
                'numNodes': 3,
                'replicationFactor': 3,
                'instanceType': instance_type['instanceTypeCode'],
                'deviceInfo': {
                    'numVolumes': 1,
                    'volumeSize': instance_type['instanceTypeDetails']['volumeDetailsList'][0]['volumeSizeGB'],
                    'storageClass': "standard"
                },
                'assignPublicIP': True,
                'useTimeSync': True,
                'enableYSQL': True,
                'enableYEDIS': False,
                'enableNodeToNodeEncrypt': True,
                'enableClientToNodeEncrypt': True,
                'enableVolumeEncryption': False,
                'ybSoftwareVersion': "2.8.0.0-b2",
                'tserverGFlags': {},
                'masterGFlags': {},
            }
        },
    ],
}

### Create Universe

Make API call to create new universe.

In [ ]:
url = f"/api/v1/customers/{customer_uuid}/universes/clusters"
conn.request("POST", url, json.dumps(new_universe), headers)
res = conn.getresponse()
pprint(json.load(res))